# Проект "Flat price prediction": Извлечение датасета

In [1]:
import os
import numpy as np
import pandas as pd
import sys
import re
import time
from bs4 import BeautifulSoup    
import requests 

In [2]:
#Используем простой звуковой индикатор для облегчения контроля за процессом
import jupyter_beeper
beeper = jupyter_beeper.Beeper()

In [3]:
#Также используем Progress Bar
from tqdm.notebook import tqdm

In [4]:
import ipython_exit

In [5]:
#Выставим опции pandas для удобства просмотра
pd.set_option('display.max_columns', 100)
pd.set_option('max_colwidth', 200)

In [6]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

Парсинг будем делать на локальной машине.

In [7]:
PATH_to_file = './data/'

Создадим словарь districs с наименованиями районов:

In [8]:
districts = ['almaty-medeuskij']
live_rooms = [1]

### Для парсинга сайта "krisha.kz" применим следующий подход:
- Применим фильтр по г. Алматы.  
- Применим фильтр по району и количеству комнат. В дальнейшем, возможно, по другим признакам.
- Попробуем прочесть доступное количество страниц.
- Дальше, как карта ляжет... )

Фильтр: Отдельно для каждого района и количества комнат:

In [9]:
'''
almaty-alatauskij
almaty-almalinskij
almaty-aujezovskij
almaty-bostandykskij
almaty-zhetysuskij
almaty-medeuskij
almaty-nauryzbajskiy
almaty-turksibskij
'''

'\nalmaty-alatauskij\nalmaty-almalinskij\nalmaty-aujezovskij\nalmaty-bostandykskij\nalmaty-zhetysuskij\nalmaty-medeuskij\nalmaty-nauryzbajskiy\nalmaty-turksibskij\n'

In [10]:
#Верхний URL:
URL_TOP = 'https://krisha.kz/'

In [11]:
district = 'almaty-medeuskij' #almaty
live_rooms = 2

In [12]:
def visit_ad(f_dict_ad_params):
#     print('Visiting ' + f_dict_ad_params['url'],
#           f_dict_ad_params['title'],
#           f_dict_ad_params['price'],
#           f_dict_ad_params['address'],
#           f_dict_ad_params['text_preview'],
#           f_dict_ad_params['owner'],
#           #f_dict_ad_params['owner_add_info'], f_dict_ad_params['residential_complex'],
#           f_dict_ad_params['stats_city'],
#           f_dict_ad_params['ad_date'],
#           f_dict_ad_params['view_count'],
#           f_dict_ad_params['highlighted'],
#           f_dict_ad_params['paid_hot'],
#           f_dict_ad_params['paid_up'],
#           f_dict_ad_params['paid_fast'],
#           f_dict_ad_params['paid_urgent'],
#           f_dict_ad_params['paid_turbo'])
    
#Заходим в объявление    
    response = requests.get(f_dict_ad_params['url'], headers={'User-Agent': 'Mozilla/5.0'})
    if response.status_code != 200:
        #raise BaseException("response code" + str(response.status_code))
        return {}
    response.encoding = 'utf-8'
    soup = BeautifulSoup(response.text, 'html.parser')    
    
    f_dict_ad_params_full = f_dict_ad_params.copy()
    
    elem_offer_container = soup.find('div', class_='offer__container')
    
    offer_short_params = ['flat.building', 'flat.floor', 'live.square', 'flat.renovation', 'flat.toilet', 'flat.balcony',
                          'flat.balcony_g', 'map.complex', 'flat.door', 'inet.type', 'live.furniture', 'flat.flooring',
                          'ceiling', 'flat.security', 'has_change', 'flat.phone', 'flat.parking', 'flat.priv_dorm']
    for param in offer_short_params:
        f_dict_ad_params_full[param] = None #Инициализация
    
    #1 left bar
    elem_offer_sidebar = elem_offer_container.find('div', class_='offer__sidebar')
    elem_offer_advert_info = elem_offer_sidebar.find('div', class_='offer__advert-info')
    elem_offer_short_description = elem_offer_advert_info.find('div', class_='offer__short-description')
    #parameters ...
    elem_offer_info_items = elem_offer_short_description.find_all('div', class_='offer__info-item')#Это список параметров
    for inf_item in elem_offer_info_items[1:]:
        f_dict_ad_params_full[inf_item['data-name']] = inf_item.find('div', class_='offer__advert-short-info').text.strip()
        
        
        
    #print('Количество elem_offer_info_items:', len(elem_offer_info_items))#Для отладки, удалить
    
    
    
    #2 right bar
    elem_offer_content = elem_offer_container.find('div', class_='offer__content')
    elem_offer_description = elem_offer_content.find('div', class_='offer__description')
    
    elem_offer_parameters = elem_offer_description.find('div', class_='offer__parameters')
    #parameters ...
    elem_offer_dls = elem_offer_parameters.find_all('dl')#Это список параметров
    for offer_dl in elem_offer_dls:
        offer_dt = offer_dl.find('dt')
        offer_dd = offer_dl.find('dd')
        f_dict_ad_params_full[offer_dt['data-name']] = offer_dd.text.strip()    
    
    
    
    
    #print('Количество elem_offer_dls:', len(elem_offer_dls))#Для отладки, удалить
    
    
    
    # Ищем тексты объявлений
    f_dict_ad_params_full['options_text'] = None
    f_dict_ad_params_full['text'] = None    
    
    elem_offer_text = elem_offer_description.find('div', class_='text')
    #2 texts
    if elem_offer_text != None:
        #1
        elem_offer_options_text = elem_offer_text.find('div', class_='a-options-text')
        if elem_offer_options_text != None:
            f_dict_ad_params_full['options_text'] = elem_offer_options_text.text.strip()
        #2
        elem_offer_long_text = elem_offer_text.find('div', class_='a-text')
        if elem_offer_long_text != None:
            f_dict_ad_params_full['text'] = elem_offer_long_text.text.strip()
    
    print(f_dict_ad_params_full)
    
    return f_dict_ad_params_full

In [13]:
def fill_ad_params(f_ad, f_page, f_highlighted, f_district, f_live_rooms):
    f_dict_ad_params = {}
    
    f_dict_ad_params['id'] = ad['data-id'] #Добавляем id
    f_dict_ad_params['page'] = f_page #Добаляем номер страницы
    f_dict_ad_params['url'] = URL_TOP + 'a/show/' + ad['data-id']
    f_dict_ad_params['district'] = f_district
    f_dict_ad_params['live_rooms'] = f_live_rooms
    
    #Дальше спускаемся вниз по иерархии содержимого объявления
    elem_ad_content = f_ad.find('div', class_= 'a-card__inc')
    elem_ad_description = elem_ad_content.find('div', class_= 'a-card__descr')
    #Пропускаем 'a-card__header'
    elem_ad_main_info = elem_ad_description.find('div', class_= 'a-card__main-info')
    elem_ad_header_left = elem_ad_main_info.find('div', class_= 'a-card__header-left')
    elem_ad_title = elem_ad_header_left.find('a', class_= 'a-card__title')
    
    f_dict_ad_params['title'] = elem_ad_title.text#Заголовок объявления
    
    elem_ad_price = elem_ad_main_info.find('div', class_= 'a-card__price')
    
    f_dict_ad_params['price'] = elem_ad_price.text.strip().replace('&nbsp;', '').replace('〒', '').replace('\xa0', '')
    
    elem_ad_header_body = elem_ad_description.find('div', class_= 'a-card__header-body')
    elem_ad_subtitle = elem_ad_header_body.find('div', class_= 'a-card__subtitle')
    
    f_dict_ad_params['address'] = elem_ad_subtitle.text.strip()
    
    elem_ad_text_preview = elem_ad_header_body.find('div', class_= 'a-card__text-preview')
    
    f_dict_ad_params['text_preview'] = elem_ad_text_preview.text.strip()
    
    #Пропускаем 'a-card__footer'
    elem_ad_owner = elem_ad_description.find('div', class_= 'a-card__owner')#Owner объявления: Хозяин или Специалист
    
    if elem_ad_owner != None:# Новостройки не берем
        elem_ad_owners_label = elem_ad_owner.find('div', class_= 'owners__label')
        f_dict_ad_params['owner'] = elem_ad_owners_label.text.strip()
    else:
        return {}
   

    elem_ad_tech_info = elem_ad_description.find('div', class_= 'a-card__card-info')
    
    elem_ad_card_stats = elem_ad_tech_info.find('div', class_= 'card-stats')
    
    elem_ad_card_stats_city = elem_ad_card_stats.find('div', class_= 'card-stats__item') #Город
    f_dict_ad_params['stats_city'] = elem_ad_card_stats_city.text
    
    elem_ad_date = elem_ad_card_stats_city.find_next_sibling() #Дата объявления
    f_dict_ad_params['ad_date'] = elem_ad_date.text.strip()
    
    elem_ad_view_count_div = elem_ad_date.find_next_sibling()
    elem_ad_view_count = elem_ad_view_count_div.find('span', class_ = 'a-view-count')#Количество просмотров
    f_dict_ad_params['view_count'] = elem_ad_view_count.text
    #print(soup.find("span", title=re.compile("RAM")).text)
    
# Значки в объявлении    
    elem_ad_paid_labels = elem_ad_tech_info.find('div', class_= 'paid-labels')    
    
    elem_ad_paid_hot = elem_ad_paid_labels.find('span', class_= 'fi-paid-hot')
    elem_ad_paid_up = elem_ad_paid_labels.find('span', class_= 'fi-paid-up')
    elem_ad_paid_fast = elem_ad_paid_labels.find('span', class_= 'fi-paid-fast')
    elem_ad_paid_urgent = elem_ad_paid_labels.find('span', class_= 'fi-paid-urgent')
    elem_ad_paid_turbo = elem_ad_paid_labels.find('span', class_= 'fi-paid-turbo')
    
    if elem_ad_paid_hot != None:
        f_dict_ad_params['paid_hot'] = 1
    else:
        f_dict_ad_params['paid_hot'] = 0
    
    if elem_ad_paid_up != None:
        f_dict_ad_params['paid_up'] = 1
    else:
        f_dict_ad_params['paid_up'] = 0
        
    if elem_ad_paid_fast != None:
        f_dict_ad_params['paid_fast'] = 1
    else:
        f_dict_ad_params['paid_fast'] = 0
        
    if elem_ad_paid_urgent != None:
        f_dict_ad_params['paid_urgent'] = 1
    else:
        f_dict_ad_params['paid_urgent'] = 0
        
    if elem_ad_paid_turbo != None:
        f_dict_ad_params['paid_turbo'] = 1
    else:
        f_dict_ad_params['paid_turbo'] = 0
        
# Помечаем выделенные объявления (которые в Highlighted Section)
    if f_highlighted:
        f_dict_ad_params['highlighted'] = 1
    else:
        f_dict_ad_params['highlighted'] = 0
    
    return f_dict_ad_params

In [14]:
url = f'{URL_TOP}prodazha/kvartiry/{district}/?das[live.rooms]={live_rooms}'

#Читаем страницу:
response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
if response.status_code != 200:
    raise BaseException("response code:" + str(response.status_code))
response.encoding = 'utf-8'
soup = BeautifulSoup(response.text, 'html.parser')

#Находим главную колонку
elem_mail_col = soup.find('div', class_='main-col')

#Находим количество объявлений по фильтру
elem_ads_count = elem_mail_col.find('div', class_='a-search-subtitle search-results-nb')
ads_count = int(elem_ads_count.text.split('Найдено ', 1)[1].split(' объявлен', 1)[0].replace('\xa0', ''))
print('Количество объявлений = ', ads_count)

#Находим количество страниц
elem_paginator = elem_mail_col.find('nav', class_='paginator')
elem_paginator_btns = elem_paginator.find_all('a', class_ = 'paginator__btn')
pages_count1 = int(elem_paginator_btns[len(elem_paginator_btns) - 2].text.strip())

if ads_count % 20 == 0:
    pages_count2 = int(ads_count / 20)
else:
    pages_count2 = int(ads_count / 20) + 1
    
pages_count = max(pages_count1, pages_count2)
print(f'Количество страниц = ', pages_count)


#Главный цикл
flats_list_global = [] # здесь соберем список списков (эл-т - словарь параметров конкретноЙ квартиры)
#pages_list = list(range(1, pages_count + 1)) # количество страниц для парсинга
pages_list = list(range(1, 1 + 1)) # количество страниц для парсинга - для отладки, удалить

with tqdm(total=len(pages_list)) as pbar:
    for p in pages_list:
        url = f'{URL_TOP}prodazha/kvartiry/{district}/?das[live.rooms]={live_rooms}&page={p}'
        response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
        if response.status_code != 200:
            raise BaseException("response code" + str(response.status_code))
        response.encoding = 'utf-8'
        soup = BeautifulSoup(response.text, 'html.parser')
        
        #Находим главную колонку
        elem_mail_col = soup.find('div', class_='main-col')        
        
        elem_main_list = elem_mail_col.find('section', class_= 'a-list a-search-list a-list-with-favs')
        
        #Ищем выделенную секцию
        elem_hilighted_section = elem_main_list.find('section', class_= 'highlighted-section')
        hs_ads_id = []#Список id объявлений из выделенной секции
        if elem_hilighted_section is not None:
            #Читаем объявления в выделенной секции
            elem_hilighted_section_ads = elem_hilighted_section.find_all('div', class_ = 'a-card')
            #Actually, the class is "a-card a-storage-live ddl_product ddl_product_link not-colored is-visible"
            print('hilighted_section_ads len = ', len(elem_hilighted_section_ads))
            for hs_ad in elem_hilighted_section_ads:#Для каждого объявления из выделенной секции
                hs_ads_id.append(hs_ad['data-id'])

            
        #Читаем обычные объявления
        elem_common_section_ads = elem_main_list.find_all('div', class_ = 'a-card')
        print('common_section_ads len = ', len(elem_common_section_ads)) 
        
        for ad in elem_common_section_ads:#Для каждого объявления из обычной секции
            if ad['data-id'] in hs_ads_id:#Если это объявление в выделенной секции 
                dict_ad_params = fill_ad_params(ad, p, True, district, live_rooms)#Заполняем словарь параметров объявления, объявление выделенное
            else:
                dict_ad_params = fill_ad_params(ad, p, False, district, live_rooms)#Заполняем словарь параметров объявления, объявление НЕ выделенное
            
            if len(dict_ad_params) > 0:
                time.sleep(0.5)
                dict_ad_params_full = visit_ad(dict_ad_params)#Посещаем страницу объявления
                if len(dict_ad_params_full) > 0:
                    flats_list_global.append(dict_ad_params_full)
                
        #Обновление ProgressBar и звуковой сигнал
        pbar.update(1)
        if p % 10 == 0:
            beeper.beep(frequency=100, secs=0.3, blocking=True)
        time.sleep(2)
        
                
print('Done.')

df = pd.DataFrame(flats_list_global)

df.head()

Количество объявлений =  1116
Количество страниц =  56


hilighted_section_ads len =  3
common_section_ads len =  20
{'id': '665595454', 'page': 1, 'url': 'https://krisha.kz/a/show/665595454', 'district': 'almaty-medeuskij', 'live_rooms': 2, 'title': '2-комнатная квартира, 41.9 м², 1/5 этаж', 'price': '23000000', 'address': 'Медеуский р-н, Тулебаева — Макатаева', 'text_preview': 'панельный дом, 1974 г.п., состояние: хорошее, санузел совмещенный, Эксклюзивное предложение от Агентства Атриум! Дом находится в тихом районе, аллея Тулебаева в 10-15 минутах ходьбы. Очень уютная квартира с теплой аурой. Документы в порядке, не в залоге, перепланов нет, ипотеку рассматриваем…', 'owner': 'Специалист', 'stats_city': 'Алматы', 'ad_date': '6 мар.', 'view_count': '', 'paid_hot': 0, 'paid_up': 1, 'paid_fast': 0, 'paid_urgent': 0, 'paid_turbo': 0, 'highlighted': 0, 'flat.building': 'панельный, 1974 г.п.', 'flat.floor': '1 из 5', 'live.square': '41.9 м², кухня — 5.98 м²', 'flat.renovation': 'хорошее', 'flat.toilet': 'совмещенный', 'flat.balcony': None, 'fla

{'id': '665558855', 'page': 1, 'url': 'https://krisha.kz/a/show/665558855', 'district': 'almaty-medeuskij', 'live_rooms': 2, 'title': '2-комнатная квартира, 44.7 м², 5/5 этаж', 'price': '17300000', 'address': 'Медеуский р-н, Маметовой — Валиханова', 'text_preview': 'панельный дом, 1974 г.п., состояние: среднее, жил. площадь 28.6 кв.м., кухня 6 кв.м., потолки 2.8м., санузел совмещенный, пустая, 219847 Продам 2х квартиру в панельном доме 1974 года постройки, неугловая, комнаты изолированные, год назад был сделан капремонт крыши, состояние квартиры среднее. Д…', 'owner': 'Специалист', 'stats_city': 'Алматы', 'ad_date': '6 мар.', 'view_count': '', 'paid_hot': 0, 'paid_up': 1, 'paid_fast': 0, 'paid_urgent': 0, 'paid_turbo': 0, 'highlighted': 0, 'flat.building': 'панельный, 1974 г.п.', 'flat.floor': '5 из 5', 'live.square': '44.7 м², жилая — 28.6 м², кухня — 6 м²', 'flat.renovation': 'среднее', 'flat.toilet': 'совмещенный', 'flat.balcony': 'балкон', 'flat.balcony_g': 'да', 'map.complex': Non

{'id': '665469743', 'page': 1, 'url': 'https://krisha.kz/a/show/665469743', 'district': 'almaty-medeuskij', 'live_rooms': 2, 'title': '2-комнатная квартира, 70 м², 4/13 этаж', 'price': '38000000', 'address': 'Медеуский р-н, Достык 44 — Жамбыла', 'text_preview': 'жил. комплекс Три Богатыря, монолитный дом, 1972 г.п., состояние: евроремонт, потолки 2.7м., санузел совмещенный, интернет ADSL, полностью меблирована, Стильная квартира в "Золотом квадрате" Алматы. Ремонт, делали со вкусом и для себя! Зал 25.2 кв. м. В зале паркет. В спальной хоро…', 'owner': 'Компания', 'stats_city': 'Алматы', 'ad_date': '6 мар.', 'view_count': '', 'paid_hot': 1, 'paid_up': 1, 'paid_fast': 0, 'paid_urgent': 0, 'paid_turbo': 0, 'highlighted': 0, 'flat.building': 'монолитный, 1972 г.п.', 'flat.floor': '4 из 13', 'live.square': '70 м²', 'flat.renovation': 'евроремонт', 'flat.toilet': 'совмещенный', 'flat.balcony': 'несколько балконов или лоджий', 'flat.balcony_g': 'да', 'map.complex': 'Три Богатыря', 'flat.door'

,id,page,url,district,live_rooms,title,price,address,text_preview,owner,stats_city,ad_date,view_count,paid_hot,paid_up,paid_fast,paid_urgent,paid_turbo,highlighted,flat.building,flat.floor,live.square,flat.renovation,flat.toilet,flat.balcony,flat.balcony_g,map.complex,flat.door,inet.type,live.furniture,flat.flooring,ceiling,flat.security,has_change,flat.phone,flat.parking,flat.priv_dorm,options_text,text
0,665595454,1,https://krisha.kz/a/show/665595454,almaty-medeuskij,2,"2-комнатная квартира, 41.9 м², 1/5 этаж",23000000,"Медеуский р-н, Тулебаева — Макатаева","панельный дом, 1974 г.п., состояние: хорошее, санузел совмещенный, Эксклюзивное предложение от Агентства Атриум! Дом находится в тихом районе, аллея Тулебаева в 10-15 минутах ходьбы. Очень уютная ...",Специалист,Алматы,6 мар.,,0,1,0,0,0,0,"панельный, 1974 г.п.",1 из 5,"41.9 м², кухня — 5.98 м²",хорошее,совмещенный,None,None,None,None,None,None,None,None,None,None,None,None,нет,None,"Эксклюзивное предложение от Агентства Атриум! Дом находится в тихом районе, аллея Тулебаева в 10-15 минутах ходьбы. Очень уютная квартира с теплой аурой. Документы в порядке, не в залоге, переплан..."
1,665388391,1,https://krisha.kz/a/show/665388391,almaty-medeuskij,2,"2-комнатная квартира, 55 м²",14575000,"Медеуский р-н, Талгарский тракт 160","жил. комплекс Нурия, монолитный дом, 2021 г.п., состояние: черновая отделка, потолки 2.7м., Отдел продаж. Купите напрямую от застройщика. Комиссия - 0%! Цена указана по Акции за 100% оплату! ЖК «Н...",Специалист,Алматы,6 мар.,,0,1,0,0,0,0,"монолитный, 2021 г.п.",10,55 м²,черновая отделка,None,лоджия,None,Нурия,металлическая,None,None,None,2.7 м,охрана,None,None,None,нет,"Пластиковые окна, улучшенная, комнаты изолированы.","Отдел продаж. Купите напрямую от застройщика. Комиссия - 0%! Цена указана по Акции за 100% оплату! \n \nЖК «Нурия» \n2 комнатная квартира, площадь 54, 9 кв. м, в улучшенной черновой отделке. Стяжк..."
2,665580652,1,https://krisha.kz/a/show/665580652,almaty-medeuskij,2,"2-комнатная квартира, 54 м²",33600000,"Медеуский р-н, проспект Абая — Желтоксан","кирпичный дом, 1961 г.п., состояние: евроремонт, санузел совмещенный, полностью меблирована, Золотой квадрат ! Исторический центр Алматы. Квартира новая ! После капитального ремонта. В квартире им...",Специалист,Алматы,6 мар.,,0,1,0,1,0,0,"кирпичный, 1961 г.п.",None,54 м²,евроремонт,совмещенный,балкон,None,None,бронированная,None,полностью меблирована,паркет,None,"домофон, видеонаблюдение",None,None,рядом охраняемая стоянка,нет,"Пластиковые окна, неугловая, улучшенная, кухня-студия, новая сантехника, счётчики, тихий двор, кондиционер.",Золотой квадрат ! Исторический центр Алматы. Квартира новая ! После капитального ремонта. В квартире имеется все необходимое для проживания. Продается сразу с мебелью. Во дворе есть охраняемая пар...
3,665395237,1,https://krisha.kz/a/show/665395237,almaty-medeuskij,2,"2-комнатная квартира, 52.5 м², 1/14 этаж",33000000,"Медеуский р-н, мкр Самал-1 33","монолитный дом, 1993 г.п., состояние: среднее, жил. площадь 30.9 кв.м., кухня 8.4 кв.м., потолки 2.85м., санузел раздельный, полностью меблирована, Продаётся 2-комнатная квартира в самом центре и ...",Хозяин недвижимости,Алматы,6 мар.,,1,1,0,0,1,0,"монолитный, 1993 г.п.",1 из 14,"52.5 м², жилая — 30.9 м², кухня — 8.4 м²",среднее,раздельный,None,None,None,None,None,полностью меблирована,линолеум,2.85 м,"домофон, видеонаблюдение, консьерж",None,None,рядом охраняемая стоянка,нет,"Пластиковые окна, тихий двор, удобно под коммерцию, комнаты изолированы.","Продаётся 2-комнатная квартира в самом центре и престижном районе Алматы - Самал-1. \nВ шаговой доступности резиденция Президента, бизнес комплекс Нурлытау, ТД Рамстор, ТД Достык Плаза, Историческ..."
4,665483515,1,https://krisha.kz/a/show/665483515,almaty-medeuskij,2,"2-комнатная квартира, 85 м², 13/20 этаж",63000000,"Медеуский р-н, мкр Самал, Достык","жил. комплекс Алматы Тауэрс, монолитный дом, 2008 г.п., состояние: хорошее, потолки 3м., санузел совмещенный, Про

In [15]:
ipython_exit.exit()

IpyExit: 

In [ ]:
brand = 'bmw'

In [ ]:
auto_list_global = [] # здесь соберем список списков (эл-т - список параметров конкретного автомобиля)
       
pages_list = list(range(1, 1201)) # количество страниц для парсинга 
with tqdm(total=len(pages_list)) as pbar:
    for i in pages_list: 
            
        url = f'https://auto.ru/moskva/cars/{brand}/used/?output_type=list&page={i}'
        response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
        if response.status_code != 200:
            raise BaseException("response code" + str(response.status_code))
        response.encoding = 'utf-8'
        soup = BeautifulSoup(response.text, 'html.parser')
        page = soup.find_all('div', class_='ListingItem-module__container')  

        for k in range(len(page)):     # проход по объявлениям на одной странице
            auto_html = page[k].find_all('meta')
            auto_list = []
            for j in range(len(auto_html)):   # обработка отдельного объявления на странице
                auto_list.append((str(auto_html[j])[15:]).split('"', 1)[0])

            auto_list_short = []
            auto_list_short.append(i) # Добавим пока номер страницы, чтобы узнать, откуда уже идут дубликаты

            # Убираем лишние параметры из списка    
            pos_out = {4,11,12,14,15,19}
            for n in range(len(auto_list)):   # удаление лишних параметров из объявления
                if n not in pos_out:
                    auto_list_short.append(auto_list[n].replace('\xa0', ' ')) 

            # Считываем пробег(расположен в другом месте)    
            all_km = page[k].find('div', class_='ListingItem-module__kmAge')
            km = str(all_km)[39:].split('<', 1)[0].replace('\xa0', ' ')
            auto_list_short.append(str(km))    # добавляем пробег
                
            auto_list_global.append(auto_list_short)    # добавляем список пар-ров автомобиля в список списков

        pbar.update(1)
        if i % 10 == 0:
            beeper.beep(frequency=340, secs=0.3, blocking=True)
        time.sleep(1.2)

In [ ]:
len(auto_list_global), len(auto_list_global[0])

In [ ]:
print(auto_list_global[:2])

Подберем в первом приближении имена столбцов в соответствии с именами в test. Как будем использовать информацию в столбце temp1, пока не понятно. 

In [ ]:
df = pd.DataFrame(auto_list_global,
                  columns=['page', 'bodyType', 'brand', 'color', 'fuelType', 'modelDate', 'model_name',
                           'numberOfDoors', 'productionDate', 'vehicleConfiguration', 'vehicleTransmission',
                           'price', 'temp1', 'engineDisplacement', 'enginePower', 'mileage'])

In [ ]:
df.head(3)

In [ ]:
df.info()

Удалим дубли:

In [ ]:
df.drop_duplicates(subset=['bodyType', 'brand', 'color', 'fuelType', 'modelDate', 'model_name', 'numberOfDoors',
                           'productionDate', 'vehicleConfiguration', 'vehicleTransmission', 'price', 'temp1',
                           'engineDisplacement', 'enginePower', 'mileage'], inplace=True)

In [ ]:
df.info()

Запишем данные в csv файл с соответствующим бренду именем:

In [ ]:
df.to_csv(PATH_to_file + brand + '.csv', index=False)

Повторив процедуру парсинга для каждой марки автомобиля из словаря, получим 12 файлов; из них потом получим один объединенный датасет train.

Считываем данные из отдельных csv и объединяем

In [ ]:
BMW = pd.read_csv(PATH_to_file + 'bmw.csv')
VOLKSWAGEN = pd.read_csv(PATH_to_file + 'volkswagen.csv')
NISSAN = pd.read_csv(PATH_to_file + 'nissan.csv')
MERCEDES = pd.read_csv(PATH_to_file + 'mercedes.csv')
TOYOTA = pd.read_csv(PATH_to_file + 'toyota.csv')
AUDI = pd.read_csv(PATH_to_file + 'audi.csv')
MITSUBISHI = pd.read_csv(PATH_to_file + 'mitsubishi.csv')
SKODA = pd.read_csv(PATH_to_file + 'skoda.csv')
VOLVO = pd.read_csv(PATH_to_file + 'volvo.csv')
HONDA = pd.read_csv(PATH_to_file + 'honda.csv')
INFINITI = pd.read_csv(PATH_to_file + 'infiniti.csv')
LEXUS = pd.read_csv(PATH_to_file + 'lexus.csv')

In [ ]:
frames = [BMW, VOLKSWAGEN, NISSAN, MERCEDES, TOYOTA, AUDI, MITSUBISHI, SKODA, VOLVO, HONDA, INFINITI, LEXUS]
train = pd.concat(frames)
train.sample(10)

In [ ]:
train.info()

Удалим пропуски, так как их мало и большое их количество находится в столбце с целевой переменной price

In [ ]:
train.dropna(inplace=True)

In [ ]:
train.info()

In [ ]:
train[train['price'].str.contains('RUB', na=False)]

In [ ]:
indices = train[train.price.str.lower().str.contains('rub', na=False)].index
train.drop(indices, inplace=True)

Преобразуем целевую переменную price в числовой формат

In [ ]:
train.price = train.price.astype('int64')

In [ ]:
train.info()

In [ ]:
train.sample(3)

Посмотрим на распределение количества объявлений по номерам страниц

In [ ]:
train[train.page < 200].page.hist()

Так как на сайте количество объявлений на странице почти фиксированное (37 - 38 объявлений) и мы удалили дубли, то можем заключить, что есть тренд к увеличению количества дублей с ростом номера страницы.

Запишем данные в итоговый csv файл

In [ ]:
train.to_csv(PATH_to_file + 'train.csv', index=False)

In [1]:
from datetime import datetime

In [2]:
now = datetime.now()

In [3]:
dt_string = now.strftime("%d-%m-%Y %H:%M:%S")
dt_string

'06-03-2021 22:35:02'

### В итоге мы получили тренировочный датасет train.csv с 26957 записей